# Исследование надежности заемщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Изучение данных

In [42]:
import pandas as pd

In [43]:
data = pd.read_csv('2.csv')

In [44]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [46]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски имеются в двух столбцах. Столбец `total_income` хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`.

In [47]:
inc_types = list(data['income_type'].unique())
for inc_type in inc_types:
    inc_data = data[data['income_type'] == inc_type]
    inc_med = inc_data['total_income'].median()
    data.loc[data['income_type'] == inc_type, 'total_income'] = data.loc[data['income_type'] == inc_type, 'total_income'].fillna(inc_med)
data.info()
    


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Обработка аномальных значений

Аномалия - отрицательное количество дней трудового стажа в столбце `days_employed`. Необходимо заменить все отрицательные значения положительными с помощью метода.

In [48]:
data['days_employed'] = data['days_employed'].abs()

In [49]:
data.groupby('income_type')['days_employed'].median()

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Так как этот столбец не понадобится для исследования, то никаких манипуляций с ним проводить не буду.

In [50]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

Вновь аномальные значения. Исправим:

In [51]:
anomal_val = [-1, 20]
for val in anomal_val:
    indexes = data[data['children'] == val].index
    data = data.drop(indexes).reset_index(drop=True)
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21397,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21398,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21399,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21400,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [52]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

In [53]:
uniq_inc = data['income_type'].unique()
for inc in uniq_inc:
    med = data[data['income_type'] == inc]['days_employed'].median()
    data.loc[data['income_type'] == inc, 'days_employed'] = data.loc[data['income_type'] == inc, 'days_employed'].fillna(med)

In [54]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [55]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [56]:
data['education'] = data['education'].str.lower()

In [57]:
data.duplicated().sum()

71

In [58]:
data.drop_duplicates().reset_index(drop=True)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21326,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21327,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21328,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21329,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


### Категоризация данных

Создам в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [59]:
def categorize_income(income):
    if income <= 30000:
        return 'E'
    elif 30001 <= income <= 50000:
        return 'D'
    elif 50001 <= income <= 200000:
        return 'C'
    elif 200001 <= income <= 1000000:
        return 'B'
    else:
        return 'A'

In [60]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [61]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создам функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [62]:
def categorize_purpose(purpose):
    if 'автомоб' in purpose:
        return 'операции с автомобилем'
    elif 'недвиж' in purpose or 'жиль' in purpose:
        return 'операции с недвижимостью'
    elif 'образ' in purpose:
        return 'получение образования'
    else:
        return 'проведение свадьбы'

In [63]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

In [64]:
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21397,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,B,операции с недвижимостью
21398,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,C,операции с автомобилем
21399,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,C,операции с недвижимостью
21400,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,B,операции с автомобилем


## Исследование данных и ответьты на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [65]:
data.groupby('children')['debt'].mean() * 100

children
0    7.512898
1    9.215442
2    9.440389
3    8.181818
4    9.756098
5    0.000000
Name: debt, dtype: float64

**Вывод:** имеется слабо выраженная связь: для любого количества детей от 1 до 4 процент невыплаты кредита держится на уровне 8 - 9%, лишь у людей без детей эта метрика опускается до 7,5%, что говорит о повышенной платежеспособности этой категории относительно других. Внимание вызывает категория с 5 детьми - процент невыплаты здесь 0%, думаю это обусловлено маленькой выборкой. В конечном итоге, если брать во внимание количество детей заёмщика, то предпочтительнее будет клиент без детей.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [66]:
data.groupby('family_status')['debt'].mean() * 100

family_status
Не женат / не замужем    9.753483
в разводе                7.064760
вдовец / вдова           6.617647
гражданский брак         9.254808
женат / замужем          7.535360
Name: debt, dtype: float64

**Вывод:** связь прослеживается. Самые ненадежные категории - не женатые / не замужние люди и пары, состоящие в гражданском браке - тут процент невыплаты держится на уровне 9,5%, за ними следуют разведённые и жентые / замужние люди - процент невозврата около 7,25%, а самые подходящие заёмщики - вдовы / вдовцы - невыплата на уровне 6,5%.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Сперва, как и в прошлых случаях, оценим процентное соотношение невыплативших кредит и общего числа заёмщиков для каждой категории:

In [67]:
data.groupby('total_income_category')['debt'].mean() * 100

total_income_category
A    8.000000
B    7.060231
C    8.460480
D    6.017192
E    9.090909
Name: debt, dtype: float64

На первый взгляд, конкретной связи тут не прослеживается, например у людей с доходом более 1.000.000 и от 50.000 до 200.000(категории A и C соотвественно) одинаковый уровень невозврата, около 8%. Самые ненадежные заёмщики, с невозвратом более чем в 9% случаев - люди из категории E, с доходом до 30.000, а самые подходящие клиенты - заёмщики из категории D(внезапно), доход которых варьируется от 30.000 до 50.000, процент невыплаты в данной категории всего 6%.

Теперь попробуем взглянуть на количество людей в каждой категории:

In [68]:
data.groupby('total_income_category')['debt'].count()

total_income_category
A       25
B     5014
C    15992
D      349
E       22
Name: debt, dtype: int64

Становится очевидным, что объективный показатель получен только для категорий C и B, так как в них достаточное для анализа количество данных. В категориях D, A, E, напротив, крайне мало данных, для того, чтобы делать какие - то выводы, поэтому наличие связи тут отследить не представляется возможным.

**Вывод:** невозможно установить наличие связи, слишком маленькая выборка.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [69]:
data.groupby('purpose_category')['debt'].mean() * 100

purpose_category
операции с автомобилем      9.328358
операции с недвижимостью    7.235622
получение образования       9.231924
проведение свадьбы          7.830552
Name: debt, dtype: float64

**Вывод:** самый большой процент невозврата у заёмов, связанных с автомобилями - 9,35%, чуть ниже этот показатель у образовательных кредитов - 9,25%. Свадебные кредиты невозвращаются почти в 8% случаев(7,9%), а заёмы на недижимость не возвращают реже всего - лишь в 7,25% случаев, что делает эти заёмы самыми предпочтительными для банка.

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

*Ответ:* помимо прочих причин по типу технической ошибки при выгрузке данных или человеческого фактора при ручном переносе данных, заёмщики могли намеренно умолчать эту информацию в собственных интересах: пропущенные данные были в столбцах про стаж и доход. Возможно, заёмщик, считая фактор дохода ключевым, не стал уточнять цифру, а может он побоялся проблем с налоговой.

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* всё дело в так называемых выбросах, которые часто бывают в количественных переменных. Выбросы очень сильно влияют на среднее значение, искажая его, а медиана к выбросам устойчива.

### Шаг 4: общий вывод.

Явно прослеживается связь между семейным положением и платежеспособностью, чуть менее явно - между платежеспособностью и количеством детей. С уровнем дохода связь установить не удалось из - за недостатка данных. Таким образом, на выбор кандидата точно стоит учитывать семейное положение и количество детей.

Судя по полученным сведениям, портрет самого надежного для банка заёмщика выглядит так:
1. Бездетный
2. Вдовец / вдова
3. Заём связан с недвижимостью

Также можно составить портрет самого нежелательного заёмщика:
1. 4 ребёнка
2. Не женат / не замужем
3. Заём связан с автомобилями
